In [1]:
import mistune
import yaml
import re
from mdrenderer import MdRenderer

In [70]:
class QuestionRenderer(MdRenderer):
    def __init__(self):
        super().__init__()
        self.current_topic = None
        self.questions = {}
        self.answer_re = re.compile(r"(\[[ |x]\])\s+(.*)")
        self.answers_started = False
            
    def header(self, text, level, raw=None):
        r = super().header(text, level, raw)
        if level == 1:
            self.current_topic = r
            self.questions[self.current_topic] = []
        elif level == 2:
            self.questions[self.current_topic].append((r, []))
        else:
            self.questions[self.current_topic][-1] = (self.questions[self.current_topic][-1][0] + '\n\n' + r, self.questions[self.current_topic][-1][1])
        return ""   
    
    def paragraph(self, text):
        r = super().paragraph(text)
        if self.answers_started:
            raise ValueError("Answers lists couldn't be interrupted by paragraphs")
        if self.current_topic is not None:
            self.questions[self.current_topic][-1] = (self.questions[self.current_topic][-1][0] + '\n\n' + r, self.questions[self.current_topic][-1][1])
        return ""    
    
    def table(self, header, body):
        r = super().table(header, body)
        self.questions[self.current_topic][-1] = (self.questions[self.current_topic][-1][0] + '\n\n' + r, self.questions[self.current_topic][-1][1])
        return ""            
    
    def _add_answer(self, text):    
        m = self.answer_re.match(text)
        if not m:
            raise ValueError("Answer does not match the expected format ({})".format(text))
        self.questions[self.current_topic][-1][1].append((m.group(1) == "[x]", m.group(2)))
        
    def list(self, text, ordered=True):
        r = ''
        self.answers_started = True
        while text:
            text, type, t = MdRenderer.get_block(text)
            if type == 'l':
                self._add_answer(t)
        #for t in self.questions[self.current_topic][-1][1]:
        #    r += (ordered and ('# ' + t[1]) or ('* ' + t[1])) + '\n'        
        self.answers_started = False
        return ""
    
renderer = QuestionRenderer()

In [73]:
markdown = mistune.Markdown()

In [75]:
filename = 'html.md'
with open('questions/{}'.format(filename), 'r') as f:    
    questions = f.read().split(r"\n---")
    for question in questions:
        print(markdown.parse(question))

<h1>HTML e Web</h1>
<hr>
<h2>Cos'è un attributo?</h2>
<table>
<thead><tr>
<th>a</th>
<th>b</th>
</tr>
</thead>
<tbody>
<tr>
<td>3</td>
<td>5</td>
</tr>
</tbody>
</table>
<ul>
<li>[x] un parametro usato all'interno dei tag per specificare informazioni aggiuntive</li>
<li>[ ] una proprietà di un'entità</li>
<li>[ ] il colore di sfondo di una pagina </li>
</ul>
<hr>
<h2>Cos'è un'ancora?</h2>
<ul>
<li>[x] un tag HTML che specifica un collegamento</li>
<li>[x] il testo associato al riferimento evidenziato nel documento</li>
<li>[ ] una ripetizione di uno o più tag</li>
</ul>
<hr>
<h2>Di cosa si occupa il World-Wide Web Consortium (W3C)</h2>
<ul>
<li>[x] di definire standard mondiali riguardanti il Web</li>
<li>[ ] l'assegnazione dei nomi di dominio</li>
<li>[ ] gestisce gli indirizzi IP statici</li>
</ul>
<hr>
<h2>Un cookie...</h2>
<ul>
<li>[x] è un informazione utilizzata da un server web per registrare informazioni su un computer client</li>
<li>[x] viene impostato dal server web e reinvi

In [66]:
renderer.questions

{'##HTML e Web\n\n': [("###Cos'è un attributo?\n\n", [])]}